In [1]:
spark

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log
4,application_1607714417077_0005,pyspark,idle,Link,Link


SparkSession available as 'spark'.

In [2]:
import hashlib
from datetime import datetime
from pyspark.sql import functions as func
from pyspark.sql.types import FloatType
import hsfs

In [3]:
# Create a connection
connection = hsfs.connection()
# Get the feature store handle for the project's feature store
fs = connection.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

## Placeholder

In [4]:
transactions_fg = fs.get_feature_group("transactions_fg", 1)

In [5]:
transactions_df = transactions_fg.read()
transactions_df.show()

+------+------+------+-------+-------+--------------+--------+--------+
|target|is_sar|source|tran_id|tx_type|tran_timestamp|alert_id|base_amt|
+------+------+------+-------+-------+--------------+--------+--------+
|  3259|     0|  1767|      1|      4|   1.4832288E9|      -1| 9405.71|
|  5141|     0|  7363|      2|      4|   1.4832288E9|      -1| 6884.54|
|  9532|     0|  7585|      3|      4|   1.4832288E9|      -1|  7968.4|
|  8792|     0|  1750|      4|      4|   1.4832288E9|      -1| 9042.67|
|  4670|     0|  9060|      5|      4|   1.4832288E9|      -1| 4692.79|
|  3861|     0|  8752|      6|      4|   1.4832288E9|      -1| 4089.65|
|  3805|     0|  9645|      7|      4|   1.4832288E9|      -1| 3055.04|
|  9280|     0|  7150|      8|      4|   1.4832288E9|      -1| 6473.45|
|  6369|     0|  1685|      9|      4|   1.4832288E9|      -1| 2583.42|
|  8255|     0|  7846|     10|      4|   1.4832288E9|      -1| 6753.04|
|  5957|     0|   878|     11|      4|   1.4832288E9|      -1| 1

## Placeholder

In [6]:
edges_td_meta = fs.create_training_dataset(name="edges_td",
                                       version=1,
                                       data_format="csv",
                                       label = ["is_sar"],   
                                       description="edges training dataset")
edges_td_meta.save(transactions_df)

### Placeholder

In [7]:
account_fg = fs.get_feature_group("account_features", 1)

In [8]:
accounts_df = account_fg.read()
accounts_df.show()

+---------+--------------+-------+---------------+
|prior_sar|tx_behavior_id|acct_id|initial_deposit|
+---------+--------------+-------+---------------+
|        0|             1|      0|       84442.19|
|        0|             1|      1|       75795.44|
|        0|             1|      2|       42057.16|
|        0|             1|      3|       25891.68|
|        0|             1|      4|       51127.47|
|        0|             1|      5|       40493.41|
|        0|             1|      6|       78379.86|
|        0|             1|      7|       30331.27|
|        0|             1|      8|        47659.7|
|        0|             1|      9|        58338.2|
|        0|             1|     10|       90811.29|
|        0|             1|     11|       50468.69|
|        0|             1|     12|       28183.78|
|        0|             1|     13|       75580.42|
|        0|             1|     14|        61836.9|
|        0|             1|     15|       25050.63|
|        0|             1|     

### Placeholder

In [9]:
source = transactions_fg.select(["source","is_sar"]).read().withColumnRenamed("source","id")
target = transactions_fg.select(["target","is_sar"]).read().withColumnRenamed("target","id")
nodes = source.union(target)
sar_nodes = nodes.where(nodes.is_sar == 1)
non_sar = nodes.where(nodes.is_sar == 0)
non_sar_nodes = non_sar.join(sar_nodes, ["id"], "leftanti")
labels = sar_nodes.union(non_sar_nodes).dropDuplicates(subset=["id"])
labels.show()

+----+------+
|  id|is_sar|
+----+------+
|3997|     1|
|5300|     1|
| 463|     1|
|8086|     1|
|7833|     1|
|1088|     0|
|6654|     0|
|9465|     0|
|7880|     0|
|6336|     0|
|1580|     0|
|1829|     0|
|7993|     0|
|7240|     0|
|7253|     0|
|8389|     0|
|4935|     0|
|6658|     0|
|6466|     0|
|8592|     0|
+----+------+
only showing top 20 rows

In [10]:
accounts_df = accounts_df.withColumnRenamed("acct_id","id")
accounts_df.show()

+---------+--------------+---+---------------+
|prior_sar|tx_behavior_id| id|initial_deposit|
+---------+--------------+---+---------------+
|        0|             1|  0|       84442.19|
|        0|             1|  1|       75795.44|
|        0|             1|  2|       42057.16|
|        0|             1|  3|       25891.68|
|        0|             1|  4|       51127.47|
|        0|             1|  5|       40493.41|
|        0|             1|  6|       78379.86|
|        0|             1|  7|       30331.27|
|        0|             1|  8|        47659.7|
|        0|             1|  9|        58338.2|
|        0|             1| 10|       90811.29|
|        0|             1| 11|       50468.69|
|        0|             1| 12|       28183.78|
|        0|             1| 13|       75580.42|
|        0|             1| 14|        61836.9|
|        0|             1| 15|       25050.63|
|        0|             1| 16|       90974.63|
|        0|             1| 17|       98278.55|
|        0|  

In [11]:
accounts_df.count()

10000

In [12]:
nodes_df = accounts_df.join(labels, ["id"])
nodes_df.count()

9998

In [13]:
node_td = fs.create_training_dataset(name="node_td",
                                       version=1,
                                       data_format="csv",   
                                       description="node training dataset")
node_td.save(nodes_df)